<a href="https://colab.research.google.com/github/SpectraGbes/Spectragbes/blob/main/MSFT%202025%20Daily%20Candlestick%20Chart.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# MSFT 2025 Daily Candlestick Chart -> PNG
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import numpy as np

csv_file = "msft_us_d.csv"

df = pd.read_csv(csv_file)

# Normalize headers to lowercase
df.columns = [c.strip().lower() for c in df.columns]

# Ensure a 'date' column exists
if 'date' not in df.columns:
    raise ValueError("CSV must contain a 'Date' column.")

df['date'] = pd.to_datetime(df['date'])
df = df.sort_values('date').reset_index(drop=True)

# Filter to full calendar year 2025
mask_2025 = (df['date'] >= "2025-01-01") & (df['date'] <= "2025-12-31")
df_2025 = df.loc[mask_2025, ['date','open','high','low','close','volume']].copy()

if df_2025.empty:
    raise ValueError("No rows found for 2025 in the provided file.")

# Compute moving averages (50D, 200D) on 2025 slice
df_2025['ma50'] = df_2025['close'].rolling(50).mean()
df_2025['ma200'] = df_2025['close'].rolling(200).mean()

# Prepare candlesticks
dates = mdates.date2num(np.array(df_2025['date'].dt.to_pydatetime()))
ohlc = list(zip(dates, df_2025['open'], df_2025['high'], df_2025['low'], df_2025['close']))

plt.style.use('seaborn-v0_8-whitegrid')
fig = plt.figure(figsize=(16, 9))
ax = fig.add_axes([0.06, 0.25, 0.93, 0.70])
ax_vol = fig.add_axes([0.06, 0.08, 0.93, 0.13], sharex=ax)

# Candlesticks (manual to avoid mplfinance dependency)
for i, (d, o, h, l, c) in enumerate(ohlc):
    color = '#2ca02c' if c >= o else '#d62728'
    # wick
    ax.vlines(d, l, h, color=color, linewidth=1)
    # body
    body_width = 0.7
    ax.add_patch(plt.Rectangle((d - body_width/2, min(o, c)),
                               body_width,
                               abs(c - o),
                               facecolor=color, edgecolor=color, linewidth=0.5))

# Moving averages
ax.plot(df_2025['date'], df_2025['ma50'], color='#1f77b4', linewidth=1.8, label='MA 50')
ax.plot(df_2025['date'], df_2025['ma200'], color='#ff7f0e', linewidth=1.8, label='MA 200')

# High/Low annotations
yr_high = df_2025['high'].max()
yr_low = df_2025['low'].min()
d_high = df_2025.loc[df_2025['high'].idxmax(), 'date']
d_low = df_2025.loc[df_2025['low'].idxmin(), 'date']
ax.scatter([d_high, d_low], [yr_high, yr_low], color='black', zorder=3, s=50)
ax.annotate(f'2025 High: {yr_high:,.2f}\n({d_high.date()})', xy=(d_high, yr_high),
            xytext=(30, 25), textcoords='offset points', fontsize=10,
            bbox=dict(boxstyle='round,pad=0.3', fc='lightgreen', alpha=0.7))
ax.annotate(f'2025 Low: {yr_low:,.2f}\n({d_low.date()})', xy=(d_low, yr_low),
            xytext=(-60, -40), textcoords='offset points', fontsize=10,
            bbox=dict(boxstyle='round,pad=0.3', fc='salmon', alpha=0.7))

# Axes formatting
ax.set_title('MSFT — Daily Candlesticks (Jan 1, 2025 → Dec 31, 2025)', fontsize=16)
ax.set_ylabel('Price (USD)', fontsize=12)
ax.legend(loc='upper left', frameon=True, fontsize=10)
ax.xaxis.set_major_locator(mdates.MonthLocator())
ax.xaxis.set_major_formatter(mdates.DateFormatter('%b'))
ax.xaxis.set_minor_locator(mdates.WeekdayLocator(byweekday=mdates.MONDAY))
ax.grid(True, linestyle='--', alpha=0.5)

# Volume bars
vol_color = '#7f7f7f'
ax_vol.bar(df_2025['date'], df_2025['volume'], width=0.8, color=vol_color)
ax_vol.set_ylabel('Volume', fontsize=12)
ax_vol.grid(True, linestyle='--', alpha=0.5)

# Tight layout and save
fig.tight_layout(rect=[0, 0.03, 1, 0.95])
fig.savefig('msft_2025_candlesticks.png', dpi=250)
plt.close(fig)

print("Saved: msft_2025_candlesticks.png")


/tmp/ipython-input-2123516868.py:33: FutureWarning: The behavior of DatetimeProperties.to_pydatetime is deprecated, in a future version this will return a Series containing python datetime objects instead of an ndarray. To retain the old behavior, call `np.array` on the result
  dates = mdates.date2num(np.array(df_2025['date'].dt.to_pydatetime()))
/tmp/ipython-input-2123516868.py:86: UserWarning: This figure includes Axes that are not compatible with tight_layout, so results might be incorrect.
  fig.tight_layout(rect=[0, 0.03, 1, 0.95])


Saved: msft_2025_candlesticks.png
